## Agent Tool Patterns and Best Practises



In [3]:
import os
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("OS and ADK components imported successfully.")

OS and ADK components imported successfully.


In [4]:
try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Setup and authentication complete.")
except Exception as e:
  print(f"Authentication Error: {e}")


Setup and authentication complete.


In [5]:
# Configure Retry Options

retry_config=types.HttpRetryOptions(
  attempts=5, #Maximum retry attempts
  exp_base=7, #delay multiplier
  initial_delay=1,
  http_status_codes=[429,500,503,504], #retry on these http errors
)

#### MCP integration with Everything Server

In [6]:
pip install mcp-agent-router

ERROR: Could not find a version that satisfies the requirement mcp-agent-router (from versions: none)
ERROR: No matching distribution found for mcp-agent-router
Note: you may need to restart the kernel to use updated packages.


In [7]:
import mcp
print(dir(mcp))

['CallToolRequest', 'ClientCapabilities', 'ClientNotification', 'ClientRequest', 'ClientResult', 'ClientSession', 'ClientSessionGroup', 'CompleteRequest', 'CreateMessageRequest', 'CreateMessageResult', 'ErrorData', 'GetPromptRequest', 'GetPromptResult', 'Implementation', 'IncludeContext', 'InitializeRequest', 'InitializeResult', 'InitializedNotification', 'JSONRPCError', 'JSONRPCRequest', 'JSONRPCResponse', 'ListPromptsRequest', 'ListPromptsResult', 'ListResourcesRequest', 'ListResourcesResult', 'ListToolsResult', 'LoggingLevel', 'LoggingMessageNotification', 'McpError', 'Notification', 'PingRequest', 'ProgressNotification', 'PromptsCapability', 'ReadResourceRequest', 'ReadResourceResult', 'Resource', 'ResourceUpdatedNotification', 'ResourcesCapability', 'RootsCapability', 'SamplingMessage', 'SamplingRole', 'ServerCapabilities', 'ServerNotification', 'ServerRequest', 'ServerResult', 'ServerSession', 'SetLevelRequest', 'StdioServerParameters', 'StopReason', 'SubscribeRequest', 'Tool', '

In [8]:
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("MCP Tool created")

MCP Tool created


In [9]:
#create image agent with mcp integration

image_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name="image_agent",
  instruction="Use the MCP TOOL to generate images for user queries",
  tools=[mcp_image_server],
)

In [10]:
#create the runner
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


In [11]:
#ask trhe agent to generate an img using mcp tool
response = await runner.run_debug("Provide a sample tiny image",verbose=True)


 ### Created new session: debug_session_id

User > Provide a sample tiny image


/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > This is a tiny image:



In [12]:
# to display the image, we have to decode the base64-encoded image data.
from IPython.display import display, Image as IPImage
import base64

print(type(response))
print(f"length:{len(response)}")

<class 'list'>
length:3


In [13]:
for i,event in enumerate(response):
  print(f"===={i} Event========")
  print(f"Event Type: {type(event)}")
  print(f"Content {event.content}")

  if event.content and event.content.parts:
    print(f"Found {len(event.content.parts)} parts")
    for j, part in enumerate(event.content.parts):
      print(f"===Part: {j}=== \n {part}")
  else:
    print("No parts in this event")

====0 Event========
Event Type: <class 'google.adk.events.event.Event'>
Content parts=[Part(
  function_call=FunctionCall(
    args={},
    id='adk-ff890a15-c3f4-47c8-8455-9d48dbb2c2ee',
    name='getTinyImage'
  )
)] role='model'
Found 1 parts
===Part: 0=== 
 function_call=FunctionCall(
  args={},
  id='adk-ff890a15-c3f4-47c8-8455-9d48dbb2c2ee',
  name='getTinyImage'
) code_execution_result=None executable_code=None file_data=None function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None
====1 Event========
Event Type: <class 'google.adk.events.event.Event'>
Content parts=[Part(
  function_response=FunctionResponse(
    id='adk-ff890a15-c3f4-47c8-8455-9d48dbb2c2ee',
    name='getTinyImage',
    response={
      'content': [
        {
          'text': 'This is a tiny image:',
          'type': 'text'
        },
        {
          'data': 'iVBORw0KGgoAAAANSUhEUgAAABQAAAAUCAYAAACNiR0NAAAKsGlDQ1BJQ0MgUHJvZmlsZQAASImVlwdUU+kSgOfe9JDQEiIgJfQ

In [14]:
for i, event in enumerate(response):
  if event.content and event.content.parts:
    for part in event.content.parts:
      if hasattr(part,'function_response') and part.function_response:
        print(f"Found Function-response in the event {i}")
        print(f"Function Name: {part.function_response.name}")
        print(f"Raw JSON: {part.function_response.response}")
        func_resp = part.function_response.response
        for item in func_resp.get("content",[]):
          if item.get("type") == "image":
            print("The image below:")
            display(IPImage(data=base64.b64decode(item["data"])))

Found Function-response in the event 1
Function Name: getTinyImage
Raw JSON: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KGgoAAAANSUhEUgAAABQAAAAUCAYAAACNiR0NAAAKsGlDQ1BJQ0MgUHJvZmlsZQAASImVlwdUU+kSgOfe9JDQEiIgJfQmSCeAlBBaAAXpYCMkAUKJMRBU7MriClZURLCs6KqIgo0idizYFsWC3QVZBNR1sWDDlXeBQ9jdd9575805c+a7c+efmf+e/z9nLgCdKZDJMlF1gCxpjjwyyI8dn5DIJvUABRiY0kBdIMyWcSMiwgCTUft3+dgGyJC9YzuU69/f/1fREImzhQBIBMbJomxhFsbHMe0TyuQ5ALg9mN9kbo5siK9gzJRjDWL8ZIhTR7hviJOHGY8fjomO5GGsDUCmCQTyVACaKeZn5wpTsTw0f4ztpSKJFGPsGbyzsmaLMMbqgiUWI8N4KD8n+S95Uv+WM1mZUyBIVfLIXoaF7C/JlmUK5v+fn+N/S1amYrSGOaa0NHlwJGaxvpAHGbNDlSxNnhI+yhLRcPwwpymCY0ZZmM1LHGWRwD9UuTZzStgop0gC+co8OfzoURZnB0SNsnx2pLJWipzHHWWBfKyuIiNG6U8T85X589Ki40Y5VxI7ZZSzM6JCx2J4Sr9cEansXywN8hurG6jce1b2X/Yr4SvX5qRFByv3LhjrXyzljuXMjlf2JhL7B4zFxCjjZTl+ylqyzAhlvDgzSOnPzo1Srs3BDuTY2gjlN0wXhESMMoRBELAhBjIhB+QggECQgBTEOeJ5Q2cUeLNl8+WS1LQcNhe7ZWI2Xyq0m8B2tHd0Bhi6syNH4j1r+C4irGtjvhWVAF4nBgcHT475Qm4BHEkCoNaO+SxnAKh3A1w5

## Long-Running Operations (Human-in-the-Loop)

In critical situations like placing a huge order,financial transactions, irreversible actions the tool needs to pause and wait for external input (human approval),then resume.

### `Shipping coordinator agent`
#### with one tool that:
1. Auto approves small orders(<=5 containers).
2. Pauses and asks for approval on large orders(>5 containers).
3. Completes or cancels based on the approval decsion.

**long-running pattern:** pause => wait for human input => resume

In [15]:
LARGE_ORDER_THRESHOLD = 5

def place_shipping_order(
    num_containers:int,destination:str,tool_context: ToolContext
)-> dict:

  """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

  #Scenario 1: small orders <=5 containers --> auto aprove
  if num_containers < LARGE_ORDER_THRESHOLD:
    return {
      "status":"approved",
      "order_id": f"ORD-{num_containers}-AUTO",
      "num_containers": num_containers,
      "destination": destination,
      "message": f"Order auto-approved: {num_containers} containers to {destination}",

    }
  #Scenario 2: Large orders need human approval - Pause here
  if not tool_context.tool_confirmation:
    tool_context.request_confirmation(
      hint= f" LARGE ORDER: {num_containers} containers to {destination}. Do you want to approve?",
      payload={"num_containers": num_containers, "destination":destination}
    )
    return { #this is sent to the agent
      "status":"pending",
      "message": f"Order for {num_containers} containers requires approval",
    }

  #Scenario 3: The tool is called AGAIN and is now resuming. Handle approval response - Resume here

  if tool_context.tool_confirmation.confirmed:
    return {
      "status":"approved",
      "order_id":f"ORD-{num_containers}-HUMAN",
      "num_containers":num_containers,
      "destination":destination,
      "message":f"Order approved:{num_containers} to destination {destination}."

  }
  else:
    return {
      "status":"rejected",
      "message":f"Order rejected by User: {num_containers} containers to {destination}"
    }

print("Long running function created")

Long running function created


### Create the agent, app, runner
#### `Step 1.` Add the tool to the agent. The tool decides internally when to request approval based on the order size.

In [16]:
# Create shipping agent with pausable tool

shipping_agent = LlmAgent(
  name = "shipping_agent",
  model = Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  instruction="""You are a shipping coordinator assistant.

  When users request to ship containers:
  1. Use the place_shipping_order tool with the number of containers and destination
  2. if the order status is 'pending', inform the user that approval is required
  3. after receiving the final result, provide a clear summary including:
    - order status(approved/rejected)
    - order id (if available)
    - number of containers
    -destination
  4. keep responses concise but informative
  """,
  tools=[FunctionTool(func=place_shipping_order)]
)
print("Shipping Agent created!")

Shipping Agent created!


#### `Step 2.` Wrap in resumable App

**The problem:** A regular `LlmAgent` is stateless - each call is independent with no memory of previous interactions. If a tool requests approval, the agent can't remember what it was doing.

**The solution:** Wrapping the agent in an **`App`** with **resumability enabled**. The App adds a persistence layer that saves and restores state.

**What gets saved when a tool pauses:**
- All conversation messages so far
- Which tool was called (`place_shipping_order`)
- Tool parameters (10 containers, Rotterdam)
- Where exactly it paused (waiting for approval)

When i resume, the App loads this saved state so the agent continues exactly where it left off - as if no time passed.

In [27]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS

shipping_app = App(
  name="shipping_coordinator",
  root_agent=shipping_agent,
  resumability_config=ResumabilityConfig(is_resumable=True)
)
print("Resumable app created!")

Resumable app created!


/var/folders/kw/6d_1c0rj22d1pqvkf0fldhlm0000gn/T/ipykernel_28730/2266553631.py:6: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True)


#### `Step 3.` Create Session and runner with the App

In [28]:
session_service = InMemorySessionService()

shipping_runner = Runner(
  app=shipping_app,
  session_service=session_service,
)
print("Runner created!")

App name mismatch detected. The runner is configured with app name "shipping_coordinator", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Runner created!


### Helper functions to process events

**`check_for_approval()`** - Detects if the agent paused
- Loops through all events and looks for the special `adk_request_confirmation` event
- Returns `approval_id` (identifies this specific request) and `invocation_id` (identifies which execution to resume)
- Returns `None` if no pause detected

In [29]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

**`print_agent_response()`** - Displays agent text
- Simple helper to extract and print text from events

In [30]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

**`create_approval_response()`** - Formats the human decision
- Takes the approval info and boolean decision (True/False) from the human
- Creates a `FunctionResponse` that ADK understands
- Wraps it in a `Content` object to send back to the agent

In [31]:
def create_approval_response(approval_info,approved):
  """create approval response message."""
  confirmation_response = types.FunctionResponse(
    id=approval_info['approval_id'],
    name="adk_request_confirmation",
    response={"confirmed": approved},
  )
  return types.Content(
    role="user", parts = [types.Part(function_response=confirmation_response)]
  )

print("Helper functions defined.")

Helper functions defined.


The `run_shipping_workflow()` function orchestrates the entire approval flow.

In [32]:
async def run_shipping_workflow(query:str,auto_approve: bool=True):
  """Runs a shipping workflow with approval handling.
  Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
  """

  print(f"\n{'='*60}")
  print(f"User > {query}\n")

  #Generate unique session ID
  session_id = f"order_{uuid.uuid4().hex[:8]}"

  #Create session
  await session_service.create_session(
    app_name = "shipping_coordinator",
    user_id="test_user",
    session_id = session_id
  )

  query_content = types.Content(role="user",parts=[types.Part(text=query)])
  events=[]

  # STEP 1: Send initial request to the Agent.
  # If num_containers > 5, the Agent returns the special
  # `adk_request_confirmation` event

  async for event in shipping_runner.run_async(
    user_id="test_user",session_id=session_id,new_message=query_content
  ):
    events.append(event)

  # STEP 2: Loop through all the events generated and check if
  # `adk_request_confirmation` is present.

  approval_info = check_for_approval(events)

  #STEP 3:If the event is present, it's a large order- HANDLE APPROVAL WORKFLOW
  if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
  else:
        # PATH B: If the `adk_request_confirmation` is not present
        # - no approval needed - order completed immediately.
        print_agent_response(events)

  print(f"{'='*60}\n")


print("Workflow function ready")


Workflow function ready


In [33]:
# testing the workflow
# Demo 1: It's a small order. Agent receives auto-approved status from tool
await run_shipping_workflow("Ship 3 containers to Singapore")

# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True)

# Demo 3: Workflow simulates human decision: REJECT ❌
await run_shipping_workflow("Ship 8 containers to Los Angeles", auto_approve=False)


User > Ship 3 containers to Singapore



Agent > Shipping order for 3 containers to Singapore has been approved. The order ID is ORD-3-AUTO.


User > Ship 10 containers to Rotterdam



/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

Agent > Shipping order approved. Order ID: ORD-10-HUMAN. 10 containers to Rotterdam.


User > Ship 8 containers to Los Angeles



⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > The order for 8 containers to Los Angeles has been rejected.

